In [1]:
from datasetcontroller.datacontroller import DataController
import os
from nnmodules.pixbylasso import LassomodPix_v2
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import time

2023-05-31 09:00:51.591192: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-31 09:00:51.691964: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
# 데이터셋 폴더 경로 설정
path = os.path.join("/workspace")
data_path = os.path.join(path,'cmaqProjectdata')

# 모델 체크포인트
proj_path = os.path.join(path,'repos','cmaqProject')
checkpoint_dir = os.path.join(proj_path,'models','training_checkpoints',"model_conmat_cmaq_4_checkpoint")
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")

# 컨트롤러
datacontroller = DataController(data_path = data_path)

# 기본 control matrix, smoke, cmaq 데이터셋 
control_matrix, _, target_arr = datacontroller.get_yearly_dataset()
X_train, X_test, y_train, y_test = train_test_split(control_matrix, target_arr, test_size=0.2, random_state=42 )

#
ctp_names_info = {}
ctp_gpd = datacontroller.get_ctp()
for i in range(len(ctp_gpd)):
    tm = ctp_gpd.loc[i,['CTP_ENG_NM','CTP_KOR_NM']].values
    ctp_names_info[tm[1]] = tm[0]
ctp_names_info['전국'] = 'korea'
ctp_names_info

/usr/local/lib/python3.8/dist-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


67 82


/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3460: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  exec(code_obj, self.user_global_ns, self.user_ns)
/workspace/repos/cmaqProject/experiment/datasetcontroller/datacontroller.py:55: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: +init=epsg:5179 +type=crs
Right CRS: EPSG:5179

  joined = gpd.sjoin(ctp_rvn_gpd, grid_data.to_crs(5179), op='contains')


{'서울특별시': 'Seoul',
 '부산광역시': 'Busan',
 '대구광역시': 'Daegu',
 '인천광역시': 'Incheon',
 '광주광역시': 'Gwangju',
 '대전광역시': 'Daejeon',
 '울산광역시': 'Ulsan',
 '세종특별자치시': 'Sejong-si',
 '경기도': 'Gyeonggi-do',
 '강원도': 'Gangwon-do',
 '충청북도': 'Chungcheongbuk-do',
 '충청남도': 'Chungcheongnam-do',
 '전라북도': 'Jeollabuk-do',
 '전라남도': 'Jellanam-do',
 '경상북도': 'Gyeongsangbuk-do',
 '경상남도': 'Gyeongsangnam-do',
 '제주특별자치도': 'Jeju-do',
 '전국': 'korea'}

In [3]:
pixlasso = LassomodPix_v2(data_path)

2023-05-31 09:00:56.061045: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-31 09:00:57.959093: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22275 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:19:00.0, compute capability: 8.6
2023-05-31 09:00:57.959744: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 19043 MB memory:  -> device: 1, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:1a:00.0, compute capability: 8.6
2023-05-31 09:00:57.960204: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localho

67 82


/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3460: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  exec(code_obj, self.user_global_ns, self.user_ns)
/workspace/repos/cmaqProject/experiment/nnmodules/pixbylasso.py:44: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: +init=epsg:5179 +type=crs
Right CRS: EPSG:5179

  joined = gpd.sjoin(ctp_rvn_gpd, grid_data.to_crs(5179), op='contains')


In [7]:
@tf.function
def train_step(input, target, epochs):
    start = time.time()
    for i, pix_index in enumerate(pixlasso._indexed_grid_point.index.tolist()):
        x, y = pix_index%82,pix_index//82
        if (i+1) % 20 == 0:
            print('.', end='', flush=True)
        
        if (i) % 200 == 0:
            if i != 0:
                print(f'Time taken for 200 grid: {time.time()-start:.2f} sec\n')
                start = time.time()
        for epoch in range(epochs):
            with tf.GradientTape() as tape:
                output = pixlasso.grid_layers[i](input)
                sub_target = target[:,x,y,:]
                l1_loss = pixlasso.lose_mse(sub_target, output)

            gradients = tape.gradient(l1_loss, pixlasso.grid_layers[i].trainable_variables)
            pixlasso.optimizers[i].apply_gradients(zip(gradients, pixlasso.grid_layers[i].trainable_variables))

epochs = 30
train_step(X_train, y_train, epochs)

..........Time taken for 200 grid: 190.19 sec

........

In [ ]:
pred = pixlasso.call(X_train)
sns.heatmap(pred[0,:,:,0][::-1])